## Find smallest combination of numbers from a list, that when added together gives a particular total, allowing a specified number of repeats for each number ##
A notebook developed to solve a task in the game Forge of Empires: have exactly X inhabitants available.

The scenario is: you are building a city. There are a range of different buildings available to you, including several different types of accommodation (increases your available population) and several types of production building (reduces your available population). You have already built some of these buildings and you also have the option to demolish some.

You are presented with a task: Have exactly X inhabitants available, where X is specified by the game (1997 in my example). How can you best achieve this?

Instructions: 
- Enter your available buildings in the list below, with the population increase that they give (negative if population is used up). Specify the maximum number of times you are willing to build (e.g. depending on availability of space) or demolish (e.g. depending on the number that you have built) each one.
- Enter the target population and the existing population in the cell below.

In [1]:
from collections import namedtuple

In [2]:
# SETUP: what actions are available (buildings that can be built or demolished) 
# and how many times are we willing to perform each one?

Bldg = namedtuple('Bldg', ['name', 'pop', 'bldMax', 'demMax'])
availableBldgs = [
    Bldg('Hus med mansardtak', pop=259, bldMax=0, demMax=5),
    Bldg('Viktorianskt hus', pop=474, bldMax=2, demMax=2),
    Bldg('Urmakare', pop=-68, bldMax=0, demMax=5),
    Bldg('Segelmakare', pop=-147, bldMax=0, demMax=1),
    Bldg('Stuga', pop=14, bldMax=5, demMax=0),
    Bldg('Pålhus', pop=22, bldMax=5, demMax=0),
    Bldg('Halmtäckthus', pop=27, bldMax=5, demMax=0),
    Bldg('Chaletstuga', pop=32, bldMax=5, demMax=0),
    Bldg('Smed', pop=-12, bldMax=5, demMax=0),
    Bldg('Jägare', pop=-28, bldMax=3, demMax=0),
]

In [3]:
# SETUP: what is the target population, what is the max number of actions, how many solutions do we want
target = 1997 - 2554 # <required population> - <existing population>
maxActions = 10 # Only consider solutions containing up to maxActions actions (increase if no solutions are found)
findMax = 5 # Stop when 5 solutions have been found (or maxActions is reached), to give a few different choices

In [4]:
# yield lists of length listlen of indices from 0 to len(reps), where each index may be repeated a variable number
# of times given by reps[index]
def getCombosWithVariableCopies(listlen, reps):
    nitems = len(reps)
    nactions = sum(reps)
    
    if listlen == 0:
        yield []
        return
    
    if listlen < 1 or listlen > nactions:
        return

    if nitems == 1:
        yield [0] * listlen
        return
    
    if listlen == 1:
        for x in range(nitems):
            if reps[x] > 0:
                yield [x]
        return
    
    if nactions == listlen:
        yield [result for idx, count in enumerate(reps) for result in [idx]*count]
        return

    for copies in range(min(reps[-1]+1, listlen+1)):
        for y in getCombosWithVariableCopies(listlen-copies, reps[:-1]):
            yield y + [nitems-1]*copies
    return

In [5]:
# # testing above function
# for g in getCombosWithVariableCopies(4, [3,1,3]):
#     print(g)
# for g in getCombosWithVariableCopies(2, [0,3,1,3,2,0]):
#     print(g)

In [6]:
# Turn list of available buildings into list of possible actions with their effect, 
actions = { (b.pop, b.name, 'Build') : b.bldMax for b in availableBldgs }
actions.update({ (-b.pop, b.name, 'Demolish') : b.demMax for b in availableBldgs })

In [7]:
# Look for combinations of actions that have the required population effect, starting with 1
# action, then try 2 actions, etc. until a combination giving the target effect is found
# continue until findMax different combinations has been found, or all combinations up to
# length maxActions have been found.

# dictionary keys and values are guaranteed to be in the same order
possChanges = list(actions.keys())
maxRepeats = list(actions.values())

found = 0
for actionCount in range(1, maxActions+1):
    for combo in getCombosWithVariableCopies(actionCount, maxRepeats):
        popChange = sum([possChanges[c][0] for c in combo])
        if popChange == target:
            found += 1
            print("Found combo:\n", '\n'.join(["\t"+str(possChanges[c]) for c in combo]))
            if found == findMax: break
    if found == findMax: break


Found combo:
 	(-28, 'Jägare', 'Build')
	(-259, 'Hus med mansardtak', 'Demolish')
	(-474, 'Viktorianskt hus', 'Demolish')
	(68, 'Urmakare', 'Demolish')
	(68, 'Urmakare', 'Demolish')
	(68, 'Urmakare', 'Demolish')
Found combo:
 	(14, 'Stuga', 'Build')
	(27, 'Halmtäckthus', 'Build')
	(-12, 'Smed', 'Build')
	(-259, 'Hus med mansardtak', 'Demolish')
	(-474, 'Viktorianskt hus', 'Demolish')
	(147, 'Segelmakare', 'Demolish')
Found combo:
 	(27, 'Halmtäckthus', 'Build')
	(27, 'Halmtäckthus', 'Build')
	(27, 'Halmtäckthus', 'Build')
	(27, 'Halmtäckthus', 'Build')
	(-259, 'Hus med mansardtak', 'Demolish')
	(-474, 'Viktorianskt hus', 'Demolish')
	(68, 'Urmakare', 'Demolish')
Found combo:
 	(22, 'Pålhus', 'Build')
	(27, 'Halmtäckthus', 'Build')
	(27, 'Halmtäckthus', 'Build')
	(32, 'Chaletstuga', 'Build')
	(-259, 'Hus med mansardtak', 'Demolish')
	(-474, 'Viktorianskt hus', 'Demolish')
	(68, 'Urmakare', 'Demolish')
Found combo:
 	(22, 'Pålhus', 'Build')
	(22, 'Pålhus', 'Build')
	(32, 'Chaletstuga', '